In [1]:
!pip install langchain
!pip install langchain_core
!pip install langchain_community
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key="<your OpenAI API key here>")

In [3]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x78baecb3f590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x78baecb97810>, root_client=<openai.OpenAI object at 0x78bb24cb51d0>, root_async_client=<openai.AsyncOpenAI object at 0x78baedcdefd0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [23]:
from typing import Optional
from math import sqrt, cos, sin
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

class CircumInput(BaseModel):
    """Inputs for basic requests calls"""
    adjacent_side: str = Field(description="The length of the adjacent side.  There is no default", default=None)
    opposite_side: str = Field(description="The length of the opposite side.  There is no default", default=None)

desc = (
    "use this tool when you need to calculate the length of an hypotenuse "
    "given two sides of a triangle  "
    "To use the tool you must provide both of the following parameters: adjacent_side, and opposite_side. provide them in separate input values not together in one string"
)

@tool("PythagorasTool", args_schema=CircumInput, return_direct=False)
def PythagorasTool(adjacent_side: str = None, opposite_side: str = None) -> float:
  """
  use this tool when you need to calculate the length of an hypotenuse
  given two sides of a triangle.
  To use the tool you must provide both of the following parameters: adjacent_side, and opposite_side. provide them in separate input values not together in one string
  Do not make up your own inputs of one or the other side is missing but ask the user to provide the missing value.
  """

  # check for the values we have been given
  print(adjacent_side)
  print(opposite_side)

  if adjacent_side and opposite_side:
    return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
  else:
    return "Could not calculate the hypotenuse of the triangle. Need both `adjacent_side`, `opposite_side`."



In [22]:
from typing import Optional
from math import sqrt, cos, sin
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

@tool("PassFailTool", return_direct=False)
def PassFailTool(course_grades : list = None) -> str:
  """
    Use this tool when you need to calculate the pass/fail grade for a set of courses
    Given a list of at least 2 numeric course grades between 0 and 100.
    To use the tool you must provide a list of numeric values together in one python list
  """

  # check for the values we have been given
  print(course_grades)

  if course_grades != None and len(course_grades) > 1:
    average = sum(course_grades) / len(course_grades)
    if average >= 65:
      return "Pass"
    else:
      return "Fail"
  else:
    return "Could not calculate the pass/fail grade. Need a list of numeric course grades between 0 and 100."



In [8]:
#tools = [retriever_tool, web_search_tool, mini_post_tool, int_compare_tool]
tools = [PassFailTool, PythagorasTool]

In [11]:
llm_with_tools = llm.bind_tools(tools)

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''
  You are a teaching assistant at CMU that helps user calculate the whether they have passed their course given a set of grades.
  Use the PassFail tool to calculate whether the average of a set of grades between 0 and 100 and whether they have passed if greater than or equal to 65.
  Use the Pythagoras tool to compute sides of a triangle given two sides.
  Do not make up your own inputs of one or the other triangle side is missing but ask the user to provide the missing value.

  Before proceeding to the next step, ensure the prior one is fully completed. Please don't mention any STEP ; just act on it.
  Be sure that you do not skip any steps.

  Step #0: For any questions not related to course grades just answer with your common knowledge

  Step #1: Determine what the set of assignment grades are
- If the user input does not specify at least two course grades then ask the user for enough so you have at least two.  Do not fill these in yourself.
- Confirmation Check: tell the user what the course grades are.

  Step #2: Determine the pass/fail grade
- Use the PassFail tool to calculate the average of the course grades and whether they have passed if greater than or equal to 65.
- Confirmation Check: Inform the user of the pass/fail decision.

  Step #3: Complete the task

            '''
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [28]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [30]:
# prompt: invoke openai agent

input1 = "adjacent side is 3 cm" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...

Invoking: `PythagorasTool` with `{'adjacent_side': '3', 'opposite_side': '5'}`


3
5
5.830951894845301The hypotenuse of the triangle with an adjacent side of 3 cm and an opposite side of 5 cm is approximately 5.83 cm.

> Finished chain.


{'input': 'adjacent side is 3 cm',
 'chat_history': [HumanMessage(content='what is the hypotenuse of a triange with opposite side 5 cm', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I couldn't calculate the hypotenuse of the triangle with just the opposite side. Could you also provide the length of the adjacent side?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='adjacent side is 3 cm', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The hypotenuse of the triangle with an adjacent side of 3 cm and an opposite side of 5 cm is approximately 5.83 cm.', additional_kwargs={}, response_metadata={})],
 'output': 'The hypotenuse of the triangle with an adjacent side of 3 cm and an opposite side of 5 cm is approximately 5.83 cm.'}